In [ ]:
using Graphs
using GraphPlot
using Combinatorics
using GraphIO

function density(g)
    ne(g) / nv(g)
end

In [ ]:
graphs = loadgraphs("datasets/graphs.lg")

# Densest At-most-k Subgraph

In [ ]:
function exact_daks(g, k)
    # Exact algorithm to find the densest at-k subgraph
    best_g = nothing
    max_d = 0
    for vec in combinations(1:nv(g), k)
        h, vmap = induced_subgraph(g, vec)
        if density(h) > max_d
            best_g = h
            max_d = density(h)
        end
    end
    best_g
end

function exact_damks(g, k)
    # Exact algorithm to find the densest at-most-k subgraph
    best_g = nothing
    max_d = 0
    for ks in 2:k
        h = daks_algo(g, ks)
        if density(h) > max_d
            max_d = density(h)
            best_g = h
        end
    end
    best_g
end

function true_densities(graphs, k)
    # Exact algorithm to find the density of DamkS
    res = Dict::{String, Float64}()
    for name, g in graphs
        res[name] = damks(g, k)
    end
    res
end

Test on graph "1"

In [ ]:
k = 8
g = exact_daks(graphs["1"], k)
gplot(g)